# 국어 문제 전처리

In [159]:
import os
import fitz  # PyMuPDF
import pytesseract
import re
import json
import glob

# Tesseract 경로 설정 (필요시)
tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

# 문제 구분을 위한 패턴 설정
question_pattern = re.compile(r"(?<![\W_])(?:\b([1-9]|[1-3][0-9]|4[0-5])\.\s*)(?!\d)")
passage_pattern = re.compile(r'\[\s*(\d+)\s*~\s*(\d+)\s*\]')  # 지문 번호 패턴

def pdf_to_json(pdf_path, output_dir):
    # 파일명에서 메타데이터 추출
    filename = os.path.basename(pdf_path)
    name_parts = filename.split(' ')
    
    # 파일명 형식 확인
    if len(name_parts) < 4:
        print(f"파일명 형식 오류: {filename}")
        return
    
    year = int(name_parts[0])
    target = name_parts[1]
    purpose = name_parts[2]
    subject = name_parts[3].split('.')[0]
    
    # 숫자만 추출
    target_number = re.findall(r'\d+', target)
    if target_number:
        target = int(target_number[0])
    
    if subject == '국어':
        subject = 0
    elif subject == '화작':
        subject = 1
    else:
        subject = 2
    
    # '수능'을 11로 변경
    if purpose == '수능':
        purpose = 11
    
    else:
        purpose_number = re.findall(r'\d+', purpose)
        if purpose_number:
            purpose = int(purpose_number[0])
    
    # 'a' 값 설정
    if purpose in [6, 9]:
        a = 1
    elif purpose == 11:
        a = 0
    else:
        a = 2

    doc = fitz.open(pdf_path)

    full_text = ""

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        images = page.get_images(full=True)

        # 이미지 추출 및 OCR 수행
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f'{filename}_page{page_num+1}_img{img_index+1}.png'
            image_path = os.path.join(output_dir, 'images', filename.split('.')[0], image_filename)
            
            # 이미지 파일로 저장
            os.makedirs(os.path.dirname(image_path), exist_ok=True)
            with open(image_path, 'wb') as img_file:
                img_file.write(image_bytes)
            
            # 이미지에서 OCR 수행
            img_text = pytesseract.image_to_string(image_path, lang="kor+eng")
            text += img_text

        full_text += text + "\n"

    lines = full_text.split('\n')
    questions = []
    question_start = None
    passages = {}
    current_passage = ""
    question_count = 1 

    # 지문 파싱
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
        passage_match = passage_pattern.search(line)
        if passage_match:
            current_passage = line
            passages[current_passage] = line
        elif current_passage:
            passages[current_passage] += " " + line

    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        match = question_pattern.match(line)
        if match:
            # 새로운 문제 시작
            if question_start is not None:
                question_end = i - 1
                question_text = ''.join(lines[question_start:question_end + 1])  # \n 제거
                question_parts = split_question_text(question_text)
                if question_parts:
                    question_number = int(question_parts.get('question_num', 0))
                    preamble, remainder = split_question_with_preamble(question_parts)
                    passage_text = find_passage(question_number, passages)
                    subject_cat = get_subject_category(subject, question_count)
                    questions.append({
                        "grade": target,
                        "yyyy": year,
                        "mm": purpose,
                        'host': a,
                        "subject_cat": subject_cat,
                        "question_num": question_number,
                        "points": extract_score(remainder),
                        "text_title": split_text_with_keyword(passage_text),
                        "text": split_text_by_question_number_2(split_text_with_keyword_2(clean_text(passage_text))),
                        "text_yn": 0 if passage_text.strip() == "" else 1,
                        "question": preamble,
                        "paragraph": clean_remainder(remainder),
                        "choice1": split_first_sentence(question_parts.get('보기1', '')),
                        "choice2": split_first_sentence(question_parts.get('보기2', '')),
                        "choice3": split_first_sentence(question_parts.get('보기3', '')),
                        "choice4": split_first_sentence(question_parts.get('보기4', '')),
                        "choice5": split_first_sentence(question_parts.get('보기5', '')),
                        "short_answer": "",
                    })
                    question_count += 1
            question_start = i

    if question_start is not None:
        question_end = len(lines)
        question_text = ''.join(lines[question_start:question_end])  # \n 제거
        question_parts = split_question_text(question_text)
        if question_parts:
            question_number = int(question_parts.get('question_num', 0))
            preamble, remainder = split_question_with_preamble(question_parts)
            passage_text = find_passage(question_number, passages)
            subject_cat = get_subject_category(subject, question_count)
            questions.append({
                "grade": target,
                "yyyy": year,
                "mm": purpose,
                'host': a,
                "subject_cat": subject_cat,
                "question_num": question_number,
                "points": extract_score(remainder),
                "text_title": split_text_with_keyword(passage_text),
                "text": split_text_by_question_number_2(split_text_with_keyword_2(clean_text(passage_text))),
                "text_yn": 0 if passage_text.strip() == "" else 1,
                "question": preamble,
                "paragraph": clean_remainder(remainder),
                "choice1": split_first_sentence(question_parts.get('보기1', '')),
                "choice2": split_first_sentence(question_parts.get('보기2', '')),
                "choice3": split_first_sentence(question_parts.get('보기3', '')),
                "choice4": split_first_sentence(question_parts.get('보기4', '')),
                "choice5": split_first_sentence(clean_text(question_parts.get('보기5', ''))),
                "short_answer": "",
            })

    output_json = os.path.join(output_dir, f'{filename.split(".")[0]}.json')

    # JSON 파일 저장
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(questions, f, ensure_ascii=False, indent=4)

    print(f"Converted {pdf_path} to {output_json}")

def get_subject_category(subject, question_count):
    if subject == 1:  # 화작
        if question_count < 35:
            return 0
        else:
            return 1
    elif subject == 2:  # 언매
        if question_count < 35:
            return 0
        else:
            return 2
    else:
        return subject

# 지문 전용 함수
def split_text_by_question_number_2(text):
    parts = re.split(r'(\d+\.\s+)', text)
    return parts[0].strip() if parts else text

def split_text_with_keyword_2(text):
    parts = re.split(r'물음에\s*답하시오.', text)
    if len(parts) < 2:
        return ""
    return parts[1].strip()

def split_question_text(text):
    parts = re.split(r'(①|②|③|④|⑤)', text)
    if len(parts) < 2:
        return None

    result = {"question_num": re.match(r'^\d+', parts[0].strip()).group()}
    result["text_title"] = re.sub(r'^\d+\.\s*', '', parts[0].strip())
    if len(parts) >= 3:
        result["보기1"] = parts[2].strip()
    if len(parts) >= 5:
        result["보기2"] = parts[4].strip()
    if len(parts) >= 7:
        result["보기3"] = parts[6].strip()
    if len(parts) >= 9:
        result["보기4"] = parts[8].strip()
    if len(parts) >= 11:
        result["보기5"] = re.sub(r'이 문제지에 관한 저작권은 한국교육과정평가원에 있습니다', '', re.split(r'국어영역', parts[10].strip())[0].strip())
    return result


def split_first_sentence(text):
    # 마침표 기준으로 분할하여 첫 문장만 반환
    sentences = text.split('.')
    return sentences[0] if sentences else text

def split_question_with_preamble(question_parts):
    question_num = question_parts.get("question_num", "")
    text_title = question_parts.get("text_title", "")
    preamble = re.sub(r'^\d+\.\s*', '', text_title)
    remainder = re.split(r'\?', text_title)
    if len(remainder) < 2:
        return [question_num + " " + preamble, ""]
    preamble = remainder[0].strip()
    remainder = " ".join(remainder[1:]).strip()
    
    return [preamble + '?', remainder]

def split_text_with_keyword(text):
    parts = re.split(r'물음에\s*답하시오', text)
    if len(parts) < 2:
        return ""
    return parts[0].strip() + ' 물음에 답하시오.'

def clean_text(text):
    text = re.sub(r'국어영역\s+\d+\s+\d+\s+\d+', '', text)
    text = re.sub(r'\[.*?\].', '', text)  # 대괄호로 감싸진 텍스트 제거
    text = re.sub(r'━{10,}', '', text)  # 연속된 ━ 문자 제거
    text = re.sub(r'\d+\s+국어 영역\s+\d+', '', text)  # "국어영역" 포함 텍스트 제거
    text = re.sub(r'\s{2,}', ' ', text)  # 연속된 공백 제거
    return text.strip()


def clean_preamble(text):
    return re.sub(r'^\d+\.\s*', '', text)

def clean_remainder(text):
    # [3점]을 제거하고 남은 텍스트 반환
    return re.sub(r'\[\d+점\]', '', text).strip()

def extract_score(text):
    # [3점]을 찾아서 배점 반환, 없으면 2
    match = re.search(r'\[(\d+)점\]', text)
    if match:
        return int(match.group(1))
    return 2

def find_passage(question_number, passages):
    for key, text in passages.items():
        numbers = re.findall(r'\d+', key)
        if len(numbers) == 2:
            start, end = int(numbers[0]), int(numbers[1])
            if start <= question_number <= end:
                return clean_text(text)
    return ""

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        pdf_to_json(pdf_file, output_directory)

# 입력 디렉토리 경로 설정
input_directory = "D:\\문제\\bigdata7-final_project--1\\문제\\국어\\고1"

# 출력 디렉토리 경로 설정
output_directory = "D:\\문제\\bigdata7-final_project--1\\문제(완)\\국어\\고1"

# 모든 PDF 파일 변환
convert_all_pdfs_in_directory(input_directory, output_directory)


Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2015 고1 11월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(완)\국어\고1\2015 고1 11월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2015 고1 3월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(완)\국어\고1\2015 고1 3월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2015 고1 6월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(완)\국어\고1\2015 고1 6월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2015 고1 9월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(완)\국어\고1\2015 고1 9월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2016 고1 11월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(완)\국어\고1\2016 고1 11월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2016 고1 3월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(완)\국어\고1\2016 고1 3월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\2016 고1 6월 국어.pdf to D:\문제\bigdata7-final_project--1\문제(완)\국어\고1\2016 고1 6월 국어.json
Converted D:\문제\bigdata7-final_project--1\문제\국어\고1\

In [124]:
import os
import fitz  # PyMuPDF
import pytesseract
import re
import json
import glob

# Tesseract 경로 설정 (필요시)
tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

# 문제 구분을 위한 패턴 설정
question_pattern = re.compile(r'(?<![\W_])(?:\b(\d+)\.\s+)(?=[윗위(<㉠[사다ⓐ])')
# option_pattern = re.compile(r'[①-⑤]\s')  # 보기 번호
passage_pattern = re.compile(r'\[\s*(\d+)\s*~\s*(\d+)\s*\]')  # 지문 번호 패턴

def pdf_to_json(pdf_path, output_dir):
    # 파일명에서 메타데이터 추출
    filename = os.path.basename(pdf_path)
    name_parts = filename.split('_')
    
    # 파일명 형식 확인
    if len(name_parts) < 4:
        print(f"파일명 형식 오류: {filename}")
        return
    
    year = int(name_parts[0])
    target = name_parts[1]
    purpose = name_parts[2]
    subject = name_parts[3].split('.')[0]
    
    # 숫자만 추출
    target_number = re.findall(r'\d+', target)
    if target_number:
        target = int(target_number[0])
    
    if subject == 'KOR':
        subject = 0
    elif subject == 'media':
        subject = 1
    else:
        subject = 2
    
    # '수능'을 11로 변경
    if purpose == 'CSAT':
        purpose = 11
    
    else:
        purpose_number = re.findall(r'\d+', purpose)
        if purpose_number:
            purpose = int(purpose_number[0])
    
    # 'a' 값 설정
    if purpose in [6, 9]:
        a = 1
    elif purpose == 11:
        a = 0
    else:
        a = 2

    doc = fitz.open(pdf_path)

    full_text = ""

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        images = page.get_images(full=True)

        # 이미지 추출 및 OCR 수행
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f'{filename}_page{page_num+1}_img{img_index+1}.png'
            image_path = os.path.join(output_dir, 'images', filename.split('.')[0], image_filename)
            
            # 이미지 파일로 저장
            os.makedirs(os.path.dirname(image_path), exist_ok=True)
            with open(image_path, 'wb') as img_file:
                img_file.write(image_bytes)
            
            # 이미지에서 OCR 수행
            img_text = pytesseract.image_to_string(image_path, lang="kor+eng")
            text += img_text

        full_text += text + "\n"

    lines = full_text.split('\n')
    questions = []
    question_start = None
    passages = {}
    current_passage = ""
    question_count = 1 

    # 지문 파싱
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
        passage_match = passage_pattern.search(line)
        if passage_match:
            current_passage = line
            passages[current_passage] = line
        elif current_passage:
            passages[current_passage] += " " + line

    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        match = question_pattern.match(line)
        if match:
            # 새로운 문제 시작
            if question_start is not None:
                question_end = i - 1
                question_text = ''.join(lines[question_start:question_end + 1])  # \n 제거
                question_parts = split_question_text(question_text)
                if question_parts:
                    question_number = int(question_parts.get('question_num', 0))
                    preamble, remainder = split_question_with_preamble(question_parts.get('text_title', ''))
                    passage_text = find_passage(question_count, passages)
                    questions.append({
                        "grade": target,
                        "yyyy": year,
                        "mm": purpose,
                        'host': a,
                        "subject_cat": subject,
                        "question_num": question_count,
                        "points": extract_score(remainder),
                        "text_title": split_text_with_keyword(passage_text),
                        "text": split_text_by_question_number_2(split_text_with_keyword_2(clean_text(passage_text))),
                        "text_yn": 0 if passage_text.strip() == "" else 1,
                        "question": clean_preamble(preamble),
                        "paragraph": clean_remainder(remainder),
                        "choice1": split_first_sentence(question_parts.get('보기1', '')),
                        "choice2": split_first_sentence(question_parts.get('보기2', '')),
                        "choice3": split_first_sentence(question_parts.get('보기3', '')),
                        "choice4": split_first_sentence(question_parts.get('보기4', '')),
                        "choice5": split_first_sentence(question_parts.get('보기5', '')),
                        "short_answer": "",
                    })
                    question_count += 1
            question_start = i

    if question_start is not None:
        question_end = len(lines)
        question_text = ''.join(lines[question_start:question_end])  # \n 제거
        question_parts = split_question_text(question_text)
        if question_parts:
            question_number = int(question_parts.get('question_num', 0))
            preamble, remainder = split_question_with_preamble(question_parts.get('text_title', ''))
            passage_text = find_passage(question_count, passages)
            questions.append({
                "grade": target,
                "yyyy": year,
                "mm": purpose,
                'host': a,
                "subject_cat": subject,
                "question_num": question_count,
                "points": extract_score(remainder),
                "text_title": split_text_with_keyword(passage_text),
                "text": split_text_by_question_number_2(split_text_with_keyword_2(clean_text(passage_text))),
                "text_yn": 0 if passage_text.strip() == "" else 1,
                "question": clean_preamble(preamble),
                "paragraph": clean_remainder(remainder),
                "choice1": split_first_sentence(question_parts.get('보기1', '')),
                "choice2": split_first_sentence(question_parts.get('보기2', '')),
                "choice3": split_first_sentence(question_parts.get('보기3', '')),
                "choice4": split_first_sentence(question_parts.get('보기4', '')),
                "choice5": split_first_sentence(question_parts.get('보기5', '')),
                "short_answer": "",
            })

    output_json = os.path.join(output_dir, f'{filename.split(".")[0]}.json')

    # JSON 파일 저장
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(questions, f, ensure_ascii=False, indent=4)

    print(f"Converted {pdf_path} to {output_json}")

# 지문 전용 함수
def split_text_by_question_number_2(text):
    parts = re.split(r'(\d+\.\s+)', text)
    return parts[0].strip() if parts else text

def split_text_with_keyword_2(text):
    parts = re.split(r'물음에\s*답하시오.', text)
    if len(parts) < 2:
        return ""
    return parts[1].strip()

def split_question_text(text):
    parts = re.split(r'(①|②|③|④|⑤)', text)
    if len(parts) < 2:
        return None

    result = {"question_num": re.match(r'^\d+', parts[0].strip()).group()}
    result["text_title"] = parts[0].strip()
    if len(parts) >= 3:
        result["보기1"] = parts[2].strip()
    if len(parts) >= 5:
        result["보기2"] = parts[4].strip()
    if len(parts) >= 7:
        result["보기3"] = parts[6].strip()
    if len(parts) >= 9:
        result["보기4"] = parts[8].strip()
    if len(parts) >= 11:
        result["보기5"] = parts[10].strip().split('국어영역')[0]
    return result


def split_first_sentence(text):
    # 마침표 기준으로 분할하여 첫 문장만 반환
    sentences = text.split('.')
    return sentences[0] if sentences else text

def split_question_with_preamble(text):
    parts = re.split(r'\?', text)
    if len(parts) < 2:
        return [text, ""]
    
    preamble = parts[0].strip()
    remainder = " ".join(parts[1:]).strip()
    
    return [preamble + '?', remainder]


def split_text_with_keyword(text):
    parts = re.split(r'물음에\s*답하시오', text)
    if len(parts) < 2:
        return ""
    return parts[0].strip() + ' 물음에 답하시오.'


def clean_text(text):
    text = re.sub(r'국어영역\s+\d+\s+\d+\s+\d+', '', text)
    text = re.sub(r'\[.*?\].', '', text)  # 대괄호로 감싸진 텍스트 제거
    text = re.sub(r'━{10,}', '', text)  # 연속된 ━ 문자 제거
    text = re.sub(r'\d+\s+국어영역\s+\d+', '', text)  # "국어영역" 포함 텍스트 제거
    text = re.sub(r'\s{2,}', ' ', text)  # 연속된 공백 제거
    return text.strip()


def clean_preamble(text):
    return re.sub(r'^\d+\.\s*', '', text)

def clean_remainder(text):
    # [3점]을 제거하고 남은 텍스트 반환
    return re.sub(r'\[\d+점\]', '', text).strip()

def extract_score(text):
    # [3점]을 찾아서 배점 반환, 없으면 2
    match = re.search(r'\[(\d+)점\]', text)
    if match:
        return int(match.group(1))
    return 2

def find_passage(question_number, passages):
    for key, text in passages.items():
        numbers = re.findall(r'\d+', key)
        if len(numbers) == 2:
            start, end = int(numbers[0]), int(numbers[1])
            if start <= question_number <= end:
                return clean_text(text)
    return ""

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        pdf_to_json(pdf_file, output_directory)

# 입력 디렉토리 경로 설정
input_directory = "D:\\문제\\bigdata7-final_project-\\문제\\국어\\고1"

# 출력 디렉토리 경로 설정
output_directory = "D:\\문제\\bigdata7-final_project-\\문제(완)\\국어\\고1"

# 모든 PDF 파일 변환
convert_all_pdfs_in_directory(input_directory, output_directory)


# 국어 평가원 x 해설지

In [111]:
import fitz  # PyMuPDF
import json
import re
import os
import glob

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

def clean_preamble(text):
    return re.sub(r'^\d+\.\s*', '', text)

def extract_passages(text):
    passage_pattern = re.compile(r'\[\d+~ \d+\]|\[\d+ ~ \d+\]')
    passages = {}
    current_passage = None

    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue
        passage_match = passage_pattern.search(line)
        if passage_match:
            current_passage = line
            passages[current_passage] = line
        elif current_passage:
            passages[current_passage] += " " + line

    return list(passages.values()), passages

def split_questions(text):
    question_pattern = re.compile(r'(\d+\.\s+\[출제의도\].*?)(?=\d+\.\s+\[출제의도\]|\d+\.\s|\Z)', re.DOTALL)
    questions = []
    question_start = None
    
    lines = text.split('\n')
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        match = question_pattern.match(line)
        if match:
            if question_start is not None:
                question_end = i - 1
                question_text = '\n'.join(lines[question_start:question_end + 1])
                questions.append(question_text)
            question_start = i

    if question_start is not None:
        question_text = '\n'.join(lines[question_start:])
        questions.append(question_text)

    return questions

def find_passage(question_number, passages):
    for key, text in passages.items():
        numbers = re.findall(r'\d+', key)
        if len(numbers) == 2:
            start, end = int(numbers[0]), int(numbers[1])
            if start <= question_number <= end:
                return clean_text(text)
    return ""

def parse_question(question, passages):
    lines = question.split('\n')
    question_number = int(lines[0].split('.')[0].strip())
    cleaned_text = clean_preamble(clean_text(question))
    passage_text = find_passage(question_number, passages)
    return {
        "question_num": question_number,
        "text_exp": cleaned_text,
        "question_exp": passage_text
    }

def pdf_to_json(pdf_path, output_path):
    full_text = extract_text_from_pdf(pdf_path)
    questions = split_questions(full_text)
    passages, passages_dict = extract_passages(full_text)
    
    json_data = []
    for question in questions:
        json_data.append(parse_question(question, passages_dict))
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    
    print(f"Converted {pdf_path} to {output_path}")

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        output_path = os.path.join(output_directory, f'{os.path.basename(pdf_file).split(".")[0]}.json')
        pdf_to_json(pdf_file, output_path)

# 예제 입력 및 출력 디렉토리
input_directory = "D:\\문제\\bigdata7-final_project--1\\해설\\국어\\고2"
output_directory = "D:\\문제\\bigdata7-final_project--1\\해설(완)\\국어\\고2"

# 지정된 디렉토리의 모든 PDF 파일을 변환
convert_all_pdfs_in_directory(input_directory, output_directory)


Converted D:\문제\bigdata7-final_project--1\해설\국어\고2\2015 고2 11월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고2\2015 고2 11월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고2\2015 고2 3월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고2\2015 고2 3월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고2\2015 고2 6월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고2\2015 고2 6월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고2\2015 고2 9월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고2\2015 고2 9월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고2\2016 고2 11월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고2\2016 고2 11월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고2\2016 고2 3월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고2\2016 고2 3월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고2\2016 고2 6월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고2\2016 고2 6월 국어 해설.json
Converted

In [101]:
import fitz  # PyMuPDF
import json
import re
import os
import glob

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text
def split_text_by_question_number_2(text):
    parts = re.split(r'(\d+\.\s+)', text)
    return parts[0].strip() if parts else text

def clean_text(text):
    return text.replace('  ', " ")

def clean_text2(text):
    return re.sub(r'\s+', ' ', text).strip()



def clean_preamble(text):
    return re.sub(r'^\d+\.\s*', '', text)

def clean_passages(text):
    return re.split(r'^\d+\.\s*', '', text)[0]

def extract_passages(text):
    # [숫자~ 숫자] 및 [숫자 ~ 숫자] 패턴에 맞는 텍스트를 추출
    passage_pattern = re.compile(r'\[\d+~ \d+\]|\[\d+ ~ \d+\]')
    passages = {}
    current_passage = None

    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue
        passage_match = passage_pattern.search(line)
        if passage_match:
            current_passage = line
            passages[current_passage] = line
        elif current_passage:
            passages[current_passage] += " " + line

    return list(passages.values()), passages

def split_questions(text):
    # [출제의도] 줄을 포함하여 텍스트를 추출하는 정규식
    question_pattern = re.compile(r'(\d+\.\s+\[출제의도\].*?)(?=\d+\.\s+\[출제의도\]|\d+\.\s|\Z)', re.DOTALL)
    questions = []
    question_start = None
    
    lines = text.split('\n')
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        match = question_pattern.match(line)
        if match:
            # 새로운 문제 시작
            if question_start is not None:
                question_end = i - 1
                question_text = '\n'.join(lines[question_start:question_end + 1])
                questions.append(question_text)
            question_start = i

    if question_start is not None:
        question_text = '\n'.join(lines[question_start:])
        questions.append(question_text)

    return questions

def find_passage(question_number, passages):
    for key, text in passages.items():
        numbers = re.findall(r'\d+', key)
        if len(numbers) == 2:
            start, end = int(numbers[0]), int(numbers[1])
            if start <= question_number <= end:
                return clean_text(text)
    return ""


def parse_question(question, passages):
    lines = question.split('\n')
    question_number = int(lines[0].split('.')[0].strip())
    # 전체 질문 텍스트를 유지
    cleaned_text = clean_preamble(clean_text(question))
    passage_text = find_passage(question_number, passages)
    return {
        "question_num": question_number,
        "explanation": clean_text2(cleaned_text + " " + split_text_by_question_number_2(passage_text))
    }

def pdf_to_json(pdf_path, output_path):
    full_text = extract_text_from_pdf(pdf_path)
    questions = split_questions(full_text)
    passages, passages_dict = extract_passages(full_text)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        for question in questions:
            json.dump(parse_question(question, passages_dict), f, ensure_ascii=False, indent=4)
            f.write('\n')  # 각 질문을 개별적으로 저장하고 줄바꿈 추가
    
    print(f"Converted {pdf_path} to {output_path}")

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        output_path = os.path.join(output_directory, f'{os.path.basename(pdf_file).split(".")[0]}.json')
        pdf_to_json(pdf_file, output_path)


# 예제 입력 및 출력 디렉토리
input_directory = "D:\\문제\\bigdata7-final_project--1\\해설\\국어\\고3"
output_directory = "D:\\문제\\bigdata7-final_project--1\\해설(완)\\국어\\고3"

# 지정된 디렉토리의 모든 PDF 파일을 변환
convert_all_pdfs_in_directory(input_directory, output_directory)



Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2019 고3 10월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2019 고3 10월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2019 고3 3월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2019 고3 3월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2019 고3 4월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2019 고3 4월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2019 고3 7월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2019 고3 7월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2020 고3 10월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2020 고3 10월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2020 고3 3월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2020 고3 3월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2020 고3 4월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2020 고3 4월 국어 해설.json
Converted

In [ ]:
import fitz  # PyMuPDF
import json
import re
import os
import glob

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

def split_text_by_question_number_2(text):
    parts = re.split(r'(\d+\.\s+)', text)
    return parts[0].strip() if parts else text

def clean_text(text):
    return text.replace('  ', " ")

def clean_text2(text):
    return re.sub(r'\s+', ' ', text).strip()

def clean_preamble(text):
    return re.sub(r'^\d+\.\s*', '', text)

def clean_passages(text):
    return re.split(r'^\d+\.\s*', '', text)[0]

def extract_passages(text):
    # [숫자~ 숫자] 및 [숫자 ~ 숫자] 패턴에 맞는 텍스트를 추출
    passage_pattern = re.compile(r'\[\d+~ \d+\]|\[\d+ ~ \d+\]')
    passages = {}
    current_passage = None

    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue
        passage_match = passage_pattern.search(line)
        if passage_match:
            current_passage = line
            passages[current_passage] = line
        elif current_passage:
            passages[current_passage] += " " + line

    return list(passages.values()), passages

def split_questions(text):
    # [출제의도] 줄을 포함하여 텍스트를 추출하는 정규식
    question_pattern = re.compile(r'(\d+\.\s+\[출제의도\].*?)(?=\d+\.\s+\[출제의도\]|\d+\.\s|\Z)', re.DOTALL)
    questions = []
    question_start = None
    
    lines = text.split('\n')
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        match = question_pattern.match(line)
        if match:
            # 새로운 문제 시작
            if question_start is not None:
                question_end = i - 1
                question_text = '\n'.join(lines[question_start:question_end + 1])
                questions.append(question_text)
            question_start = i

    if question_start is not None:
        question_text = '\n'.join(lines[question_start:])
        questions.append(question_text)

    return questions

def find_passage(question_number, passages):
    for key, text in passages.items():
        numbers = re.findall(r'\d+', key)
        if len(numbers) == 2:
            start, end = int(numbers[0]), int(numbers[1])
            if start <= question_number <= end:
                return clean_text(text)
    return ""

def parse_question(question, passages):
    lines = question.split('\n')
    question_number = int(lines[0].split('.')[0].strip())
    # 전체 질문 텍스트를 유지
    cleaned_text = clean_preamble(clean_text(question))
    passage_text = find_passage(question_number, passages)
    return {
        "question_num": question_number,
        "explanation": clean_text2(cleaned_text + " " + split_text_by_question_number_2(passage_text))
    }

def extract_subject_from_filename(filename):
    name_parts = filename.split(' ')
    
    # 파일명 형식 확인
    if len(name_parts) < 4:
        print(f"파일명 형식 오류: {filename}")
        return None
    
    subject = name_parts[3].split('.')[0]
    
    return subject

def parse_answers(text, subject):
    # 두 가지 형식을 만족하는 정규 표현식
    pattern = re.compile(r'(\d{1,2})[.\s]*([①②③④⑤])')
    matches = pattern.findall(text)
    
    # 정답 변환 맵
    answer_map = {'①': 1, '②': 2, '③': 3, '④': 4, '⑤': 5}
    
    # JSON 형식으로 변환
    answers = [{"question_num": int(num), "multiple_answer": answer_map[ans], "count": idx + 1, "subject": subject} for idx, (num, ans) in enumerate(matches)]
    
    # 조건에 따라 필터링
    if len(answers) > 45:
        if subject == "화작":
            answers = [ans for ans in answers if ans["count"] <= 45]
        elif subject == "언매":
            answers = [ans for ans in answers if (ans["count"] <= 34) or (ans["count"] >= 46 and ans["count"] <= 56)]
    
    return answers

def pdf_to_json(pdf_path, output_path):
    # 파일명에서 subject 추출
    filename = os.path.basename(pdf_path)
    subject = extract_subject_from_filename(filename)
    
    if subject is None:
        return
    
    # PDF에서 텍스트 추출
    text = extract_text_from_pdf(pdf_path)
    
    # 정답 파싱 및 subject 추가
    answers = parse_answers(text, subject)
    
    # JSON 형식 데이터 생성
    json_data = {
        "answers": answers
    }
    
    # JSON 파일로 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    
    print(f"Converted {pdf_path} to {output_path}")

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        output_path = os.path.join(output_directory, f'{os.path.basename(pdf_file).split(".")[0]}.json')
        pdf_to_json(pdf_file, output_path)

# 예제 입력 및 출력 디렉토리
input_directory = "D:\\문제\\bigdata7-final_project--1\\해설\\국어\\고3"
output_directory = "D:\\문제\\bigdata7-final_project--1\\해설(완)\\국어\\고3"

# 지정된 디렉토리의 모든 PDF 파일을 변환
convert_all_pdfs_in_directory(input_directory, output_directory)

# 예제 입력 및 출력 디렉토리
input_directory_answers = "D:\\문제\\bigdata7-final_project--1\\해설\\국어\\고3"
output_directory_answers = "D:\\문제\\bigdata7-final_project--1\\정답(완)\\국어\\고3"

# 지정된 디렉토리의 모든 PDF 파일을 변환
convert_all_pdfs_in_directory(input_directory_answers, output_directory_answers)


# 국어 평가원 x 정답지

In [103]:
import fitz  # PyMuPDF
import os
import json
import re

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

def extract_subject_from_filename(filename):
    name_parts = filename.split(' ')
    
    # 파일명 형식 확인
    if len(name_parts) < 4:
        print(f"파일명 형식 오류: {filename}")
        return None
    
    subject = name_parts[3].split('.')[0]
    
    return subject

def parse_answers(text, subject):
    # 두 가지 형식을 만족하는 정규 표현식
    pattern = re.compile(r'(\d{1,2})[.\s]*([①②③④⑤])')
    matches = pattern.findall(text)
    
    # 정답 변환 맵
    answer_map = {'①': 1, '②': 2, '③': 3, '④': 4, '⑤': 5}
    
    # JSON 형식으로 변환
    answers = [{"question_num": int(num), "multiple_answer": answer_map[ans], "count": idx + 1, "subject": subject} for idx, (num, ans) in enumerate(matches)]
    
    # 조건에 따라 필터링
    if len(answers) > 45:
        if subject == "화작":
            answers = [ans for ans in answers if ans["count"] <= 45]
        elif subject == "언매":
            answers = [ans for ans in answers if (ans["count"] <= 34) or (ans["count"] >= 46 and ans["count"] <= 56)]
    
    return answers

def pdf_to_json(pdf_path, output_path):
    # 파일명에서 subject 추출
    filename = os.path.basename(pdf_path)
    subject = extract_subject_from_filename(filename)
    
    if subject is None:
        return
    
    # PDF에서 텍스트 추출
    text = extract_text_from_pdf(pdf_path)
    
    # 정답 파싱 및 subject 추가
    answers = parse_answers(text, subject)
    
    # JSON 형식 데이터 생성
    json_data = []
    for answer in answers:
        json_data.append({
            "subject_cat": subject,  # 과목
            "question_num": answer["question_num"],  # 질문 번호
            "multiple_answer": answer["multiple_answer"],  # 정답
        })
    
    # JSON 파일로 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    
    print(f"Converted {pdf_path} to {output_path}")

def process_all_pdfs_in_directory(input_directory, output_directory):
    # PDF 파일 목록 가져오기
    pdf_files = [f for f in os.listdir(input_directory) if f.endswith('.pdf')]
    
    # 출력 디렉토리 생성 (존재하지 않으면)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    # 각 PDF 파일 처리
    converted_count = 0
    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_directory, pdf_file)
        output_path = os.path.join(output_directory, f'{os.path.splitext(pdf_file)[0]}.json')
        
        # PDF에서 텍스트 추출 및 JSON 파일로 저장
        pdf_to_json(pdf_path, output_path)
        converted_count += 1
    
    print(f"총 {converted_count}개의 PDF 파일을 변환했습니다.")

# 예제 입력 및 출력 디렉토리
input_directory = "D:\\문제\\bigdata7-final_project--1\\해설\\국어\\고3"
output_directory = "D:\\문제\\bigdata7-final_project--1\\정답(완)\\국어\\고3"

# 지정된 디렉토리의 모든 PDF 파일을 변환
process_all_pdfs_in_directory(input_directory, output_directory)


Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2019 고3 10월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\정답(완)\국어\고3\2019 고3 10월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2019 고3 3월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\정답(완)\국어\고3\2019 고3 3월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2019 고3 4월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\정답(완)\국어\고3\2019 고3 4월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2019 고3 7월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\정답(완)\국어\고3\2019 고3 7월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2020 고3 10월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\정답(완)\국어\고3\2020 고3 10월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2020 고3 3월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\정답(완)\국어\고3\2020 고3 3월 국어 해설.json
Converted D:\문제\bigdata7-final_project--1\해설\국어\고3\2020 고3 4월 국어 해설.pdf to D:\문제\bigdata7-final_project--1\정답(완)\국어\고3\2020 고3 4월 국어 해설.json
Converted

# 영어 기본 해설지 코드

In [56]:

import fitz  # PyMuPDF
import json
import re
import os
import glob

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text
def split_text_by_question_number_2(text):
    parts = re.split(r'(\d+\.\s+)', text)
    return parts[0].strip() if parts else text

def clean_text(text):
    return text.replace('  ', " ")

def clean_text2(text):
    return re.sub(r'\s+', ' ', text).strip()



def clean_preamble(text):
    return re.sub(r'^\d+\.\s*', '', text)

def clean_passages(text):
    return re.split(r'^\d+\.\s*', '', text)[0]

def extract_passages(text):
    # [숫자~ 숫자] 및 [숫자 ~ 숫자] 패턴에 맞는 텍스트를 추출
    passage_pattern = re.compile(r'\[\d+~ \d+\]|\[\d+ ~ \d+\]')
    passages = {}
    current_passage = None

    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue
        passage_match = passage_pattern.search(line)
        if passage_match:
            current_passage = line
            passages[current_passage] = line
        elif current_passage:
            passages[current_passage] += " " + line

    return list(passages.values()), passages

def split_questions(text):
    # [출제의도] 줄을 포함하여 텍스트를 추출하는 정규식
    question_pattern = re.compile(r'(\d+\.\s+\[.*?)(?=\d+\.\s+\[|\d+\.\s+|\Z)', re.DOTALL)

    questions = []
    question_start = None
    
    lines = text.split('\n')
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        match = question_pattern.match(line)
        if match:
            # 새로운 문제 시작
            if question_start is not None:
                question_end = i - 1
                question_text = '\n'.join(lines[question_start:question_end + 1])
                questions.append(question_text)
            question_start = i

    if question_start is not None:
        question_text = '\n'.join(lines[question_start:])
        questions.append(question_text)

    return questions

def find_passage(question_number, passages):
    for key, text in passages.items():
        numbers = re.findall(r'\d+', key)
        if len(numbers) == 2:
            start, end = int(numbers[0]), int(numbers[1])
            if start <= question_number <= end:
                return clean_text(text)
    return ""


def parse_question(question, passages):
    lines = question.split('\n')
    question_number = int(lines[0].split('.')[0].strip())
    # 전체 질문 텍스트를 유지
    cleaned_text = clean_preamble(clean_text(question))
    passage_text = find_passage(question_number, passages)
    return {
        "question_num": question_number,
        "explanation": clean_text2(cleaned_text + " " + split_text_by_question_number_2(passage_text))
    }

def pdf_to_json(pdf_path, output_path):
    full_text = extract_text_from_pdf(pdf_path)
    questions = split_questions(full_text)
    passages, passages_dict = extract_passages(full_text)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        for question in questions:
            json.dump(parse_question(question, passages_dict), f, ensure_ascii=False, indent=4)
            f.write('\n')  # 각 질문을 개별적으로 저장하고 줄바꿈 추가
    
    print(f"Converted {pdf_path} to {output_path}")

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        output_path = os.path.join(output_directory, f'{os.path.basename(pdf_file).split(".")[0]}.json')
        pdf_to_json(pdf_file, output_path)


# 예제 입력 및 출력 디렉토리
input_directory = "C:\\Users\\BIG3-04\\Desktop\\특수반\\영어\\해설"
output_directory = "C:\\Users\\BIG3-04\\Desktop\\특수반\\영어\\해설(완2)"

# 지정된 디렉토리의 모든 PDF 파일을 변환
convert_all_pdfs_in_directory(input_directory, output_directory)


Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2017학년도 고2 11월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2017학년도 고2 11월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2018학년도 고2 11월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2018학년도 고2 11월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2018학년도 고2 3월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2018학년도 고2 3월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2018학년도 고2 9월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2018학년도 고2 9월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2019학년도 고2 3월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2019학년도 고2 3월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2020학년도 고3 6월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2020학년도 고3 6월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2021학년도 고1 11월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2021학년도 고1 11월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2021학년도 고2 11월 해설.pdf to C:\Users\BIG3-

# 영어 해설지 2019년 7월 전용

In [59]:
import fitz  # PyMuPDF
import json
import re
import os
import glob

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

def split_text_by_question_number_2(text):
    parts = re.split(r'(\d+\.\s+)', text)
    return parts[0].strip() if parts else text

def clean_text(text):
    return text.replace('  ', " ")

def clean_text2(text):
    return re.sub(r'\s+', ' ', text).strip()

def clean_preamble(text):
    return re.sub(r'^\d+\.\s*', '', text)

def clean_passages(text):
    return re.split(r'^\d+\.\s*', '', text)[0]

def extract_passages(text):
    # [숫자~ 숫자] 및 [숫자 ~ 숫자] 패턴에 맞는 텍스트를 추출
    passage_pattern = re.compile(r'\[\d+~ \d+\]|\[\d+ ~ \d+\]')
    passages = {}
    current_passage = None

    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue
        passage_match = passage_pattern.search(line)
        if passage_match:
            current_passage = line
            passages[current_passage] = line
        elif current_passage:
            passages[current_passage] += " " + line

    return list(passages.values()), passages

def split_questions(text):
    # [출제의도] 줄을 포함하여 텍스트를 추출하는 정규식
    question_pattern = re.compile(r'(\s?\d+~?\d*\.\s+\[.*?)(?=\s?\d+~?\d*\.\s+\[|\s?\d+~?\d*\.\s+|\Z)', re.DOTALL)

    questions = []
    question_start = None
    
    lines = text.split('\n')
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        match = question_pattern.match(line)
        if match:
            # 새로운 문제 시작
            if question_start is not None:
                question_end = i - 1
                question_text = '\n'.join(lines[question_start:question_end + 1])
                questions.append(question_text)
            question_start = i

    if question_start is not None:
        question_text = '\n'.join(lines[question_start:])
        questions.append(question_text)

    return questions

def find_passage(question_number, passages):
    for key, text in passages.items():
        numbers = re.findall(r'\d+', key)
        if len(numbers) == 2:
            start, end = int(numbers[0]), int(numbers[1])
            if start <= question_number <= end:
                return clean_text(text)
    return ""

def parse_question(question, passages):
    lines = question.split('\n')
    question_number_part = lines[0].split('.')[0].strip()
    
    if '~' in question_number_part:
        start_num, end_num = map(int, question_number_part.split('~'))
        question_numbers = range(start_num, end_num + 1)
    else:
        question_numbers = [int(question_number_part)]
    
    cleaned_text = clean_preamble(clean_text(question))
    explanations = {}
    
    for question_number in question_numbers:
        passage_text = find_passage(question_number, passages)
        explanations[question_number] = clean_text2(cleaned_text + " " + split_text_by_question_number_2(passage_text))
    
    return explanations

def pdf_to_json(pdf_path, output_path):
    full_text = extract_text_from_pdf(pdf_path)
    questions = split_questions(full_text)
    passages, passages_dict = extract_passages(full_text)
    
    result = []
    
    for question in questions:
        explanations = parse_question(question, passages_dict)
        for question_num, explanation in explanations.items():
            result.append({
                "question_num": question_num,
                "explanation": explanation
            })
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
    
    print(f"Converted {pdf_path} to {output_path}")

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        output_path = os.path.join(output_directory, f'{os.path.basename(pdf_file).split(".")[0]}.json')
        pdf_to_json(pdf_file, output_path)


# 예제 입력 및 출력 디렉토리
input_directory = "C:\\Users\\BIG3-04\\Desktop\\특수반\\영어\\해설"
output_directory = "C:\\Users\\BIG3-04\\Desktop\\특수반\\영어\\해설(완2)"


# 지정된 디렉토리의 모든 PDF 파일을 변환
convert_all_pdfs_in_directory(input_directory, output_directory)


Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2017학년도 고2 11월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2017학년도 고2 11월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2018학년도 고2 11월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2018학년도 고2 11월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2018학년도 고2 3월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2018학년도 고2 3월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2018학년도 고2 9월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2018학년도 고2 9월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2019학년도 고2 3월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2019학년도 고2 3월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2020학년도 고3 6월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2020학년도 고3 6월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2021학년도 고1 11월 해설.pdf to C:\Users\BIG3-04\Desktop\특수반\영어\해설(완2)\2021학년도 고1 11월 해설.json
Converted C:\Users\BIG3-04\Desktop\특수반\영어\해설\2021학년도 고2 11월 해설.pdf to C:\Users\BIG3-

# 영어 문제 파싱

In [191]:
import os
import json
import re
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = []
    for page in reader.pages:
        text.append(page.extract_text())
    return "\n".join(text)


def clean_question_text(text):
    # 특정 패턴 "영어영역 1-2자리 숫자 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1-2자리 숫자" 삭제
    text = re.sub(r'영어영역\s+\d+\s+\d+\s+\d+', '', text)
    text = re.sub(r'\[.*?\].', '', text)  # 대괄호로 감싸진 텍스트 제거
    text = re.sub(r'━{10,}', '', text)  # 연속된 ━ 문자 제거
    text = re.sub(r'\d+\s+영어영역\s+\d+', '', text)  # "국어영역" 포함 텍스트 제거
    text = re.sub(r'영어영역\d{3}', '', text)
    text = re.sub(r'영어영역\s\d\s\d{2}', '', text)
    text = re.sub(r'\s{2,}', ' ', text)  # 연속된 공백 제거
    return text.strip()

def split_text_alternating(text):
    lines = text.split('\n')
    left_text = []
    right_text = []
    for i, line in enumerate(lines):
        if i % 2 == 0:
            left_text.append(line.strip())
        else:
            right_text.append(line.strip())
    return '\n'.join(left_text), '\n'.join(right_text)

def pdf_to_json_alternating(pdf_path, output_directory):
    # PDF에서 텍스트 추출
    text = extract_text_from_pdf(pdf_path)
    
    # 텍스트 내의 개행 문자를 공백으로 대체
    cleaned_text = clean_question_text(text)
    
    # 텍스트를 좌우로 분할
    left_text, right_text = split_text_alternating(cleaned_text)
    
    # 좌우 JSON 파일로 저장
    base_filename = os.path.splitext(os.path.basename(pdf_path))[0]
    left_output_path = os.path.join(output_directory, f'{base_filename}_left.json')

    
    with open(left_output_path, 'w', encoding='utf-8') as f:
        json.dump({"text": left_text}, f, ensure_ascii=False, indent=4)
        
    print(f"Converted {pdf_path} to {left_output_path}")

def process_all_pdfs_in_directory(input_directory, output_directory):
    # PDF 파일 목록 가져오기
    pdf_files = [f for f in os.listdir(input_directory) if f.endswith('.pdf')]
    
    # 출력 디렉토리 생성 (존재하지 않으면)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    # 각 PDF 파일 처리
    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_directory, pdf_file)
        pdf_to_json_alternating(pdf_path, output_directory)

# 예제 입력 및 출력 디렉토리
input_directory = "D:\\문제\\bigdata7-final_project--1\\문제\\영어\\고3"
output_directory = "D:\\문제\\bigdata7-final_project--1\\문제(완1)\\영어\\고3"

# 지정된 디렉토리의 모든 PDF 파일을 변환
process_all_pdfs_in_directory(input_directory, output_directory)


Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 10월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완1)\영어\고3\2019학년도 대학수학능력시험 10월 문제_left.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 3월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완1)\영어\고3\2019학년도 대학수학능력시험 3월 문제_left.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 4월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완1)\영어\고3\2019학년도 대학수학능력시험 4월 문제_left.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 6월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완1)\영어\고3\2019학년도 대학수학능력시험 6월 문제_left.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 7월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완1)\영어\고3\2019학년도 대학수학능력시험 7월 문제_left.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 9월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완1)\영어\고3\2019학년도 대학수학능력시험 9월 문제_left.json
Converted D:\문제\bigdata7-final_project--1\문제

In [77]:
import os
import fitz  # PyMuPDF
import pytesseract
import re
import json
import glob

# Tesseract 경로 설정 (필요시)
tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

# 문제 구분을 위한 패턴 설정
question_pattern = re.compile(r'\s*(\d+)\.\s*')
passage_pattern = re.compile(r'\[\s*(\d+)\s*~\s*(\d+)\s*\]')  # 지문 번호 패턴
question_answer_pattern = re.compile(r'(고르시오\.|답하시오\.)')

def pdf_to_json(pdf_path, output_dir):
    # 파일명에서 메타데이터 추출
    filename = os.path.basename(pdf_path)
    name_parts = filename.split(' ')
    
    if len(name_parts) < 4:
        print(f"파일명 형식 오류: {filename}")
        return
    
    year = name_parts[0]
    target = '공통' if name_parts[1] == '대학수학능력시험' else name_parts[1]
    purpose = name_parts[2].split('월')[0]
    subject = 3 if name_parts[3].split('.')[0] == '문제' else name_parts[3].split('.')[0]

    if purpose == '수능':
        purpose = 11
    else:
        purpose_number = re.findall(r'\d+', purpose)
        if purpose_number:
            purpose = int(purpose_number[0])
    
    a = 1 if purpose in [6, 9] else 0 if purpose == 11 else 2

    # PDF에서 텍스트와 이미지 추출
    doc = fitz.open(pdf_path)
    full_text = ""

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        images = page.get_images(full=True)

        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f'{filename}_page{page_num+1}_img{img_index+1}.png'
            image_path = os.path.join(output_dir, 'images', filename.split('.')[0], image_filename)
            
            os.makedirs(os.path.dirname(image_path), exist_ok=True)
            with open(image_path, 'wb') as img_file:
                img_file.write(image_bytes)
            
            img_text = pytesseract.image_to_string(image_path, lang="kor+eng")
            text += img_text

        full_text += text + "\n"

    lines = full_text.split('\n')
    questions = []
    question_start = None
    passages = {}
    current_passage = ""
    question_count = 1 

    # 지문 파싱
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
        passage_match = passage_pattern.search(line)
        if passage_match:
            current_passage = line
            passages[current_passage] = line
        elif current_passage:
            passages[current_passage] += " " + line

    # 문제 파싱
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        match = question_pattern.match(line)
        if match:
            if question_start is not None:
                question_end = i - 1
                question_text = ''.join(lines[question_start:question_end + 1])  
                question_parts = split_question_text(question_text)
                if question_parts:
                    question_number = int(question_parts.get('question_num', 0))
                    preamble, remainder = split_question_with_preamble(question_parts.get('text_title', ''))
                    passage_text = find_passage(question_count, passages)
                    questions.append({
                        "grade": target,
                        "yyyy": year,
                        "mm": purpose,
                        'host': a,
                        "subject_cat": subject,
                        "question_num": question_count,
                        "points": extract_score(remainder),
                        "text_title": split_text_with_keyword(passage_text),
                        "text": split_text_by_question_number_2(split_text_with_keyword_2(clean_text(passage_text))),
                        "text_yn": 0 if passage_text.strip() == "" else 1,
                        "question": preamble,
                        "paragraph": clean_remainder(remainder),
                        "choice1": split_first_sentence(question_parts.get('보기1', '')),
                        "choice2": split_first_sentence(question_parts.get('보기2', '')),
                        "choice3": split_first_sentence(question_parts.get('보기3', '')),
                        "choice4": split_first_sentence(question_parts.get('보기4', '')),
                        "choice5": split_first_sentence(question_parts.get('보기5', '')),
                        "multiple_answer": "",
                        "short_answer": "",
                    })
                    question_count += 1
            question_start = i

    if question_start is not None:
        question_end = len(lines) - 1
        question_text = ''.join(lines[question_start:question_end + 1])
        question_parts = split_question_text(question_text)
        if question_parts:
            question_number = int(question_parts.get('question_num', 0))
            preamble, remainder = split_question_with_preamble(question_parts.get('text_title', ''))
            passage_text = find_passage(question_count, passages)
            questions.append({
                "grade": target,
                "yyyy": year,
                "mm": purpose,
                'host': a,
                "subject_cat": subject,
                "question_num": question_count,
                "points": extract_score(remainder),
                "text_title": split_text_with_keyword(passage_text),
                "text": split_text_by_question_number_2(split_text_with_keyword_2(clean_text(passage_text))),
                "text_yn": 0 if passage_text.strip() == "" else 1,
                "question": preamble,
                "paragraph": clean_remainder(remainder),
                "choice1": split_first_sentence(question_parts.get('보기1', '')),
                "choice2": split_first_sentence(question_parts.get('보기2', '')),
                "choice3": split_first_sentence(question_parts.get('보기3', '')),
                "choice4": split_first_sentence(question_parts.get('보기4', '')),
                "choice5": split_first_sentence(question_parts.get('보기5', '')),
                "multiple_answer": "",
                "short_answer": "",
            })

    output_json = os.path.join(output_dir, f'{filename.split(".")[0]}.json')
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(questions, f, ensure_ascii=False, indent=4)

    print(f"Converted {pdf_path} to {output_json}")

# 지문 전용 함수
def split_text_by_question_number_2(text):
    parts = re.split(r'(\d+\.\s+)', text)
    return parts[0].strip() if parts else text

def split_text_with_keyword_2(text):
    parts = re.split(question_answer_pattern, text)
    if len(parts) < 3:
        return ""
    return parts[2].strip()

def split_question_text(text):
    parts = re.split(r'(①|②|③|④|⑤)', text)
    if len(parts) < 2:
        return None

    result = {"question_num": re.match(r'^\d+', parts[0].strip()).group()}
    result["text_title"] = parts[0].strip()
    if len(parts) >= 3:
        result["보기1"] = parts[2].strip()
    if len(parts) >= 5:
        result["보기2"] = parts[4].strip()
    if len(parts) >= 7:
        result["보기3"] = parts[6].strip()
    if len(parts) >= 9:
        result["보기4"] = parts[8].strip()
    if len(parts) >= 11:
        result["보기5"] = re.split(r'영어영역|고|이제|이 문제|\[|\d+', parts[10].strip())[0].strip()
    return result

def split_first_sentence(text):
    sentences = re.split(r'[.?\n]', text)
    return sentences[0] if sentences else text

def split_question_with_preamble(text):
    parts = re.split(r'\?', text)
    if len(parts) < 2:
        return [text, ""]
    preamble = parts[0].strip()
    remainder = " ".join(parts[1:]).strip()
    return [preamble + '?', remainder]

def split_text_with_keyword(text):
    parts = re.split(question_answer_pattern, text)
    if len(parts) < 3:
        return text.strip()
    return parts[0].strip() + " " + parts[1]

def clean_text(text):
    text = re.sub(r'\[.*?\].', '', text)  # 대괄호로 감싸진 텍스트 제거
    text = re.sub(r'━{10,}', '', text)  # 연속된 ━ 문자 제거
    text = re.sub(r'\s{2,}', ' ', text)  # 연속된 공백 제거
    return text.strip()

def clean_preamble(text):
    return re.sub(r'^\d+\.\s*', '', text)

def clean_remainder(text):
    return re.sub(r'\[\d+점\]', '', text).strip()

def extract_score(text):
    match = re.search(r'\[(\d+)점\]', text)
    return int(match.group(1)) if match else 2

def find_passage(question_number, passages):
    for key, text in passages.items():
        numbers = re.findall(r'\d+', key)
        if len(numbers) == 2:
            start, end = int(numbers[0]), int(numbers[1])
            if start <= question_number <= end:
                return clean_text(text)
    return ""

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        pdf_to_json(pdf_file, output_directory)

input_directory = "D:\\문제\\bigdata7-final_project--1\\문제\\영어\\고3"
output_directory = "D:\\문제\\bigdata7-final_project--1\\문제(완)\\영어\\고3"

convert_all_pdfs_in_directory(input_directory, output_directory)


Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 10월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 10월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 3월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 3월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 4월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 4월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 6월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 6월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 7월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 7월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 9월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 9월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 수능 문제.pdf to

In [165]:
import os
import json
import re
from PyPDF2 import PdfReader

# 문제 구분을 위한 패턴 설정
question_pattern = re.compile(r"\d+\.\s.*?①.*?(?=\d+\.\s|$)", re.DOTALL)
passage_pattern = re.compile(r"\[\s*([1-9]|[1-3][0-9]|4[0-5])\s*~\s*([1-9]|[1-3][0-9]|4[0-5])\s*\]")
question_answer_pattern = re.compile(r"(것은\?|고르시오\.|문장은\?)")

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = []
    for page in reader.pages:
        text.append(page.extract_text())
    return "\n".join(text)

def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

def pdf_to_json(pdf_path, output_dir):
    # 파일명에서 메타데이터 추출
    filename = os.path.basename(pdf_path)
    name_parts = filename.split(' ')
    
    # 파일명 형식 확인
    if len(name_parts) < 4:
        print(f"파일명 형식 오류: {filename}")
        return
    
    year = int(name_parts[0][:4])
    target = 3 if name_parts[1] == '대학수학능력시험' else name_parts[1]  # 학년변경
    purpose = name_parts[2].split('월')[0]
    subject = '공통' if name_parts[3].split('.')[0] == '문제' else name_parts[3].split('.')[0]

    if purpose == '수능':
        purpose = 11
    else:
        purpose_number = re.findall(r'\d+', purpose)
        if purpose_number:
            purpose = int(purpose_number[0])
            
    a = 1 if purpose in [6, 9] else 0 if purpose == 11 else 2

    # PDF에서 텍스트 추출
    text = extract_text_from_pdf(pdf_path)
    
    # 텍스트 내의 개행 문자를 공백으로 대체
    cleaned_text = clean_text(text)

    # 문제와 지문을 텍스트에서 분리
    questions = split_questions(cleaned_text)
    passages = split_passages(cleaned_text)

    # 각 문제에 메타데이터 추가
    for i, question in enumerate(questions):
        question.update({
            "grade": target,
            "yyyy": year,
            "mm": purpose,
            "host": a,
            "question_cat": subject,
            "question_num": i + 1,
        })
        # points 필드가 question 데이터의 point 값으로 설정됨
        question["points"] = question.pop("point")
        
        # text 필드 분할 및 보기 설정
        question_text = question.pop("text")
        question_text, choices_text = split_text_with_keyword(question_text)
        question["bbb"] = question_text
        question["보기"] = choices_text

        if "보기1" in question:
            question["choice1"] = question.pop("보기1")
        if "보기2" in question:
            question["choice2"] = question.pop("보기2")
        if "보기3" in question:
            question["choice3"] = question.pop("보기3")
        if "보기4" in question:
            question["choice4"] = question.pop("보기4")
        if "보기5" in question:
            question["choice5"] = question.pop("보기5")

    # JSON 파일 저장
    output_json = os.path.join(output_dir, f'{filename.split(".")[0]}.json')
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump({"questions": questions, "passages": passages}, f, ensure_ascii=False, indent=4)

    print(f"Converted {pdf_path} to {output_json}")

def split_questions(text):
    matches = question_pattern.findall(text)
    questions = []

    for i, match in enumerate(matches):
        question_text = re.sub(r'\s+', ' ', match.strip())  # 줄바꿈 문자 제거 및 다중 공백을 단일 공백으로 변경
        question_data = split_question_text(question_text)  # 텍스트 분할 및 "question_data" 추출
        if question_data:
            point = 3 if '[3점]' in match else 2  # "point" 값 설정
            question_data["point"] = point
            questions.append(question_data)

    return questions

def split_question_text(text):
    parts = re.split(r'(①|②|③|④|⑤)', text)
    if len(parts) < 2:
        return None

    result = {"text": parts[0].strip()}
    if len(parts) >= 3:
        result["보기1"] = parts[2].strip()
    if len(parts) >= 5:
        result["보기2"] = parts[4].strip()
    if len(parts) >= 7:
        result["보기3"] = parts[6].strip()
    if len(parts) >= 9:
        result["보기4"] = parts[8].strip()
    if len(parts) >= 11:
        result["보기5"] = re.split(r'영어영역|고|이제|이 문제|\[|\d+', parts[10].strip())[0].strip()
    return result

def split_text_with_keyword(text):
    parts = re.split(question_answer_pattern, text)
    if len(parts) < 3:
        parts = re.split(r"\d+\.", text)
        if len(parts) < 2:
            return text.strip(), ""
        return parts[0].strip(), parts[1].strip()
    return parts[0].strip() + " " + parts[1], parts[2].strip()

def split_passages(text):
    passages = []
    for match in passage_pattern.finditer(text):
        start, end = map(int, match.groups())
        passage_text = ""
        end_pattern = re.compile(rf"\b{end}\.\s")
        end_match = end_pattern.search(text, match.end())
        if end_match:
            passage_text = text[match.end():end_match.start()].strip()
        else:
            passage_text = text[match.end():].strip()
        passages.append({
            "text": re.sub(r'\s+', ' ', passage_text)  # 줄바꿈 문자 제거 및 다중 공백을 단일 공백으로 변경
        })
    return passages

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = [f for f in os.listdir(input_directory) if f.endswith('.pdf')]
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_directory, pdf_file)
        pdf_to_json(pdf_path, output_directory)

# 입력 디렉토리 경로 설정
input_directory = "D:\\문제\\bigdata7-final_project--1\\문제\\영어\\고3"
output_directory = "D:\\문제\\bigdata7-final_project--1\\문제(완)\\영어\\고3"

# 모든 PDF 파일 변환
convert_all_pdfs_in_directory(input_directory, output_directory)


Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 10월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 10월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 3월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 3월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 4월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 4월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 6월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 6월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 7월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 7월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 9월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 9월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 수능 문제.pdf to

KeyboardInterrupt: 

In [192]:
import os
import fitz  # PyMuPDF
import pytesseract
import json
import re
import glob

# Tesseract 경로 설정 (필요시)
tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

# 문제 구분을 위한 패턴 설정
question_pattern = re.compile(r"\d+\.\s.*?①.*?(?=\d+\.\s|$)", re.DOTALL)
passage_pattern = re.compile(r"\[\s*([1-9]|[1-3][0-9]|4[0-5])\s*~\s*([1-9]|[1-3][0-9]|4[0-5])\s*\]")
question_answer_pattern = re.compile(r"(것은\?|고르시오\.|문장은\?)")

def pdf_to_json(pdf_path, output_dir):
    # 파일명에서 메타데이터 추출
    filename = os.path.basename(pdf_path)
    name_parts = filename.split(' ')
    
    # 파일명 형식 확인
    if len(name_parts) < 4:
        print(f"파일명 형식 오류: {filename}")
        return
    
    year = int(name_parts[0][:4])
    target = 3 if name_parts[1] == '대학수학능력시험' else name_parts[1]  # 학년변경
    purpose = name_parts[2].split('월')[0]
    subject = '공통' if name_parts[3].split('.')[0] == '문제' else name_parts[3].split('.')[0]

    if purpose == '수능':
        purpose = 11
    else:
        purpose_number = re.findall(r'\d+', purpose)
        if purpose_number:
            purpose = int(purpose_number[0])
            
    a = 1 if purpose in [6, 9] else 0 if purpose == 11 else 2
    doc = fitz.open(pdf_path)
    extracted_data = []

    # 이미지 저장 디렉토리 생성
    image_dir = os.path.join(output_dir, 'images', filename.split('.')[0])
    os.makedirs(image_dir, exist_ok=True)

    full_text = ""
    all_page_images = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        images = page.get_images(full=True)

        page_images = []

        # 이미지 추출 및 OCR 수행
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f'{filename}_page{page_num+1}_img{img_index+1}.png'
            image_path = os.path.join(image_dir, image_filename)
            
            # 이미지 파일로 저장
            with open(image_path, 'wb') as img_file:
                img_file.write(image_bytes)
            
            # 이미지에서 OCR 수행
            img_text = pytesseract.image_to_string(image_path, lang="kor+eng")
            text += img_text
            page_images.append(image_path)

        full_text += text  # 줄바꿈 문자 제거
        all_page_images.append(page_images)

    # 문제와 지문을 텍스트에서 분리
    questions = split_questions(full_text)
    passages = split_passages(full_text)

    # 각 문제에 메타데이터 추가
    for i, question in enumerate(questions):
        question.update({
            "grade": target,
            "yyyy": year,
            "mm": purpose,
            "host": a,
            "question_cat": subject,
            "question_num": i + 1,
        })
        # points 필드가 question 데이터의 point 값으로 설정됨
        question["points"] = question.pop("point")
        
        # text 필드 분할 및 보기 설정
        question_text = question.pop("text")
        question_text, choices_text = split_text_with_keyword(question_text)
        question["bbb"] = question_text
        question["보기"] = choices_text

        if "보기1" in question:
            question["choice1"] = question.pop("보기1")
        if "보기2" in question:
            question["choice2"] = question.pop("보기2")
        if "보기3" in question:
            question["choice3"] = question.pop("보기3")
        if "보기4" in question:
            question["choice4"] = question.pop("보기4")
        if "보기5" in question:
            question["choice5"] = question.pop("보기5")

    # JSON 파일 저장
    output_json = os.path.join(output_dir, f'{filename.split(".")[0]}.json')
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump({"questions": questions, "passages": passages}, f, ensure_ascii=False, indent=4)

    print(f"Converted {pdf_path} to {output_json}")

def split_questions(text):
    matches = question_pattern.findall(text)
    questions = []

    for i, match in enumerate(matches):
        question_text = re.sub(r'\s+', ' ', match.strip())  # 줄바꿈 문자 제거 및 다중 공백을 단일 공백으로 변경
        question_data = split_question_text(question_text)  # 텍스트 분할 및 "question_data" 추출
        if question_data:
            point = 3 if '[3점]' in match else 2  # "point" 값 설정
            question_data["point"] = point
            questions.append(question_data)

    return questions

def split_question_text(text):
    parts = re.split(r'(①|②|③|④|⑤)', text)
    if len(parts) < 2:
        return None

    result = {"text": parts[0].strip()}
    if len(parts) >= 3:
        result["보기1"] = parts[2].strip()
    if len(parts) >= 5:
        result["보기2"] = parts[4].strip()
    if len(parts) >= 7:
        result["보기3"] = parts[6].strip()
    if len(parts) >= 9:
        result["보기4"] = parts[8].strip()
    if len(parts) >= 11:
        result["보기5"] = re.split(r'영어영역|고|이제|이 문제|\[|\d+', parts[10].strip())[0].strip()
    return result

def split_text_with_keyword(text):
    parts = re.split(question_answer_pattern, text)
    if len(parts) < 3:
        parts = re.split(r"\d+\.", text)
        if len(parts) < 2:
            return text.strip(), ""
        return parts[0].strip(), parts[1].strip()
    return parts[0].strip() + " " + parts[1], parts[2].strip()

def split_passages(text):
    passages = []
    for match in passage_pattern.finditer(text):
        start, end = map(int, match.groups())
        passage_text = ""
        end_pattern = re.compile(rf"\b{end}\.\s")
        end_match = end_pattern.search(text, match.end())
        if end_match:
            passage_text = text[match.end():end_match.start()].strip()
        else:
            passage_text = text[match.end():].strip()
        passages.append({
            "text": re.sub(r'\s+', ' ', passage_text)  # 줄바꿈 문자 제거 및 다중 공백을 단일 공백으로 변경
        })
    return passages

def convert_all_pdfs_in_directory(input_directory, output_directory):
    pdf_files = glob.glob(os.path.join(input_directory, "*.pdf"))
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for pdf_file in pdf_files:
        pdf_to_json(pdf_file, output_directory)

# 입력 디렉토리 경로 설정
input_directory = "D:\\문제\\bigdata7-final_project--1\\문제\\영어\\고3"
output_directory = "D:\\문제\\bigdata7-final_project--1\\문제(완)\\영어\\고3"

# 모든 PDF 파일 변환
convert_all_pdfs_in_directory(input_directory, output_directory)

Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 10월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 10월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 3월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 3월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 4월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 4월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 6월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 6월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 7월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 7월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 9월 문제.pdf to D:\문제\bigdata7-final_project--1\문제(완)\영어\고3\2019학년도 대학수학능력시험 9월 문제.json
Converted D:\문제\bigdata7-final_project--1\문제\영어\고3\2019학년도 대학수학능력시험 수능 문제.pdf to

In [ ]:
import os
import openai
from PIL import Image
import pytesseract

# OpenAI API 키 설정
openai.api_key = os.getenv('OPENAI_API_KEY')

def extract_text_from_image(image_path):
    # 이미지 열기
    image = Image.open(image_path)
    # OCR로 텍스트 추출
    text = pytesseract.image_to_string(image, lang='kor+eng')  # 한글과 영어 모두 추출하도록 설정
    return text

def get_chatgpt_response(prompt):
    response = openai.Completion.create(
        engine="gpt-4",  # 원하는 엔진 선택
        prompt=prompt,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7
    )
    return response.choices[0].text.strip()

def main(image_path):
    # 이미지에서 텍스트 추출
    extracted_text = extract_text_from_image(image_path)
    print("Extracted Text:\n", extracted_text)
    
    # 추출된 텍스트를 ChatGPT API로 처리
    chatgpt_response = get_chatgpt_response(extracted_text)
    print("ChatGPT Response:\n", chatgpt_response)

if __name__ == "__main__":
    image_path = "path_to_your_image_file.png"  # 처리할 이미지 파일 경로
    main(image_path)





In [ ]:
import pandas as pd
df1 = pd.read_json("D:\문제\bigdata7-final_project--1\해설(완)\국어\고3\2019 고3 3월 국어 해설.json")
df2 = pd.read_json("D:\문제\bigdata7-final_project--1\정답(완)\국어\고3\2019 고3 3월 국어 해설.json")